In [183]:
# %%
import sys
sys.path.append('../')


from typing import Final

# from helpers.ploting import plotBargraph, plotSubBarGraphs
from helpers.graphs import intersection_of_graphs

# from helpers.operator_actions import __getTrueAndNuisanceSources, __getFinalOperatorAlarmRelationGraph, getTrueAndNuisanceSourceNames
from helpers.alarms import  filterAlarmData, getDFWithCommonSourcesInAllMonths, loadAlarmsData, loadOperatorData
from helpers import alarms
import time
from datetime import datetime, timedelta
from multiprocessing import Pool
import networkx as nx

# from multiprocessing import Pool

class Action2Alarm: # a class which relate operator actions to most suitable corresponding alarms 
    def __init__(self):
        # print("class Created")
        pass

    def __checkAction_OnAlarm(self, action, alarm,time_to_react): # Ahmet Can time_to_react filter # Done
        if action["EventTime"] >= alarm["StartTime"] and action["EventTime"] <= alarm["EndTime"] and timedelta.total_seconds(action["EventTime"]-alarm['StartTime']) <=time_to_react:
            return True
        else:
            return False

    def __addOrUpdateEdge(self,g, e): # e = (op, alarm) # Done
        if g.has_edge(*e) == True:
            g.edges[e]["weight"] += 1
        else:
            g.add_edge(*e, weight=1)
    
    def __updateNode(self, g, node):  # using partial thats why node as last arg
        # if operatorf == True:
        #     node = "Operator->" + node
        # g.nodes[node]["size"] += 0.01
        g.nodes[node]["count"] += 1
        
    def constructSingleAlarmsActionsGraph(self,args): # Done  # case 3
        df_alarms  = args[0] # alarms
        df_actions = args[1] # operator actions
        time_to_react = args[2] # time to react 
        # edge_drop_factor = args[3]  
        g = nx.DiGraph()  # Directed graph
    

        # ---------- Adding Nodes----------------
        alarm_nodes =   [alarm for alarm in df_alarms['SourceName'].unique()]
        action_nodes = ["Operator->"+action for action in df_actions['SourceName'].unique()]     

        _ = [g.add_node(node, size=0, count=0) for node in alarm_nodes+action_nodes]        
        _ = [self.__updateNode(g,node) for node in df_alarms["SourceName"]]
        _ = [self.__updateNode(g,"Operator->"+node) for node in df_actions["SourceName"]]

        # -------- Adding Edges -----------------
        alarms = df_alarms.to_dict(orient="records")
        actions = df_actions.to_dict(orient="records")

        edges = [("Operator->"+action["SourceName"], alarm["SourceName"]) for action in actions for alarm in alarms if self.__checkAction_OnAlarm(action, alarm,time_to_react)]
        
        _ = [self.__addOrUpdateEdge(g,e) for e in edges]

        # if edge_drop_factor > -1:
        #     g = filterEdges(g, edge_drop_factor=edge_drop_factor)

        return g

    def __constructMultipleAlarmsActionsGraphs(self,df_alarms, df_actions,time_to_react,edge_drop_factor):
        # graphs = [self.constructSingleAlarmsActionsGraph((df_alarms[df_alarms["Year-Month"].isin([month])],df_actions[df_actions["Year-Month"].isin([month])],time_to_react,edge_drop_factor)) for month in  df_alarms["Year-Month"].unique()]
        graphs = []

        for month in  df_alarms["Year-Month"].unique():
            inputs = (df_alarms[df_alarms["Year-Month"].isin([month])],df_actions[df_actions["Year-Month"].isin([month])],time_to_react,edge_drop_factor)
            temp_g =  self.constructSingleAlarmsActionsGraph(inputs)
            graphs.append(temp_g)

        return graphs
    
    def getOperatorAlarmRelationGraph(self, df_alarms, df_actions, min_graphs_intersection_filter,time_to_react,edge_drop_factor):
        # print(">> Finding relation between operator action and alarm")
        graphs = self.__constructMultipleAlarmsActionsGraphs(
            df_alarms, df_actions,time_to_react,edge_drop_factor)
            
        assert len(graphs) == len(list(df_alarms["Year-Month"].unique()))
        
        # print(f"\n\n\n\n>> Taking intersection of {len(graphs)} sub-graphs")
        G = intersection_of_graphs(graphs, min_graphs_intersection_filter) # min_graphs_intersection_filter is the gamma filter from the paper 

        return G

In [184]:
# check mps here

In [185]:

# /home/waris/Github/research/tupras-analysis/data/processed/alarms/final/final-all-months-alarms.csv
"""  Lodading the Data and Preprocessing """

PATH = "/Users/hamzagorgulu/Desktop/thesis/Waris Final/tupras/data/"

alarm_file_path = PATH + "final-all-months-alarms-with-day-filtered.csv"
op_action_file_path = PATH + "final-all-month-actions-filtered.csv"

start = time.time()

df_temp_alarms_main =loadAlarmsData(file_path=alarm_file_path)
df_temp_actions = loadOperatorData(file_path=op_action_file_path)



""" Common Sources in all months. Try it but can be skipped. """
# df_main_alarms = getDFWithCommonSourcesInAllMonths(df_main_alarms)

""" Chaning name 2 alias for alarm data but skipping it """
# source2Alias, alias2source = convertSourceNamesToAlias(df_main_alarms)

print("Total Time to load the data ", time.time()-start)
df_temp_alarms_main
df_temp_alarms_main.info()

Total Time to load the data  70.20949721336365
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12222300 entries, 0 to 12222299
Data columns (total 11 columns):
 #   Column       Dtype         
---  ------       -----         
 0   MachineName  object        
 1   SourceName   object        
 2   Message      object        
 3   Condition    object        
 4   MessageType  object        
 5   StartTime    datetime64[ns]
 6   EndTime      datetime64[ns]
 7   EndMessage   object        
 8   TimeDelta    float64       
 9   Year-Month   object        
 10  Day          int64         
dtypes: datetime64[ns](2), float64(1), int64(1), object(7)
memory usage: 1.0+ GB


In [186]:
df_temp_alarms_main.drop_duplicates(keep=False,inplace=True)
df_temp_alarms_main.info()
df_temp_actions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12221979 entries, 0 to 12222299
Data columns (total 11 columns):
 #   Column       Dtype         
---  ------       -----         
 0   MachineName  object        
 1   SourceName   object        
 2   Message      object        
 3   Condition    object        
 4   MessageType  object        
 5   StartTime    datetime64[ns]
 6   EndTime      datetime64[ns]
 7   EndMessage   object        
 8   TimeDelta    float64       
 9   Year-Month   object        
 10  Day          int64         
dtypes: datetime64[ns](2), float64(1), int64(1), object(7)
memory usage: 1.1+ GB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104700 entries, 0 to 104699
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   MachineName  104700 non-null  object        
 1   SourceName   104700 non-null  object        
 2   EventTime    104700 non-null  datetime64[ns]
 3   Year-Month

In [187]:
df_temp_actions.drop_duplicates(keep=False, inplace=True)
df_temp_actions

,MachineName,SourceName,EventTime,Year-Month
0,PHD47B,47FIC017,2020-01-02 14:11:30,"(2020, 1)"
1,PHD47B,47FIC017,2020-01-02 14:46:05,"(2020, 1)"
2,PHD47B,47FIC013,2020-01-02 08:54:53,"(2020, 1)"
3,PHD47B,47FIC021,2020-01-02 08:20:01,"(2020, 1)"
4,PHD47B,47FIC013,2020-01-02 09:09:30,"(2020, 1)"
...,...,...,...,...
104695,PHD47B,48FFX005,2020-02-26 05:46:24,"(2020, 2)"
104696,PHD47B,47TIC927E,2020-02-28 19:09:33,"(2020, 2)"
104697,PHD47B,48TIC034,2020-02-26 00:16:03,"(2020, 2)"
104698,PHD47B,47TIC927E,2020-02-28 19:41:28,"(2020, 2)"


In [188]:
common_df = df_temp_alarms_main[df_temp_alarms_main['SourceName'].isin(df_temp_actions['SourceName'].unique())]
common_df_sources = common_df['SourceName'].unique()
common_df_sources 

array(['48TIC2026', '47FLI-032', '47PDI1746', '47FIC1535', '47PIC1755',
       '47LIC1555', '48XL001-ANN', '47PIC1507', '47LIC1520', '47LIC1506',
       '47FIC1522A', '48AIC001', '47CEMS-SO2N', '48LI013', '47LIC002',
       '47LIC015', '47FI1575', '47PI1710', '47LIC1501', '47LIC1518',
       '48LI011', '48TIC034', '47TI1521', '47FI046', '47LIC1509',
       '47TI1506', '47FIC1551', '47LIC1513', '47FIC1516', '47TI416',
       '47LI024', '47FIC1543', '47TI414G', '48LIC009', '48LI015',
       '48LIC002', '47TI1769', '47FIC1534', '47TI414I', '47LIC1522',
       '47TI1551', '47TIC1535', '47PIC1505', '47FI1509', '47FIC035',
       '47TI1533', '47FIC1520', '47PDIC1759', '47LIC1507', '47LIC1505',
       '47TI1540', '47TI1705', '47FIC1523', '47TIC1551', '47FIC1503',
       '47PIC1560', '47TI414L', '47PIC1712', '47FIC1504', '47TI1722',
       '47TI414P', '47FIC1502', '47FIC012', '47TI1507', '47TI1505',
       '47FIC016', '47FIC038', '47FIC021', '47TI1724', '47FIC020',
       '47FIC1515', '47PI100

In [189]:
df_temp_alarms = df_temp_alarms_main

In [190]:
print("Alarms >",df_temp_alarms["SourceName"].value_counts())

print("Actions >", df_temp_actions["SourceName"].value_counts())

Alarms > 48TIC2026          7162316
47TI931A           3285328
47FLI-032           294110
48PAL007-ANN        250472
47TI003A            124399
                    ...   
47HCD-080HH-ANN          1
47HSD-030HH-ANN          1
47HCD-061HH-ANN          1
47HCD-063HH-ANN          1
47FI033C                 1
Name: SourceName, Length: 1090, dtype: int64
Actions > 47TIC274           5269
47FIC1535          4985
47PIC027           4674
47FIC013           4069
47FIC1563          3903
                   ... 
47APC_RCAS_SHDF       1
47FIC1503_DSET        1
47FIC1504_DSET        1
47FIC1505_DSET        1
47APC_F21_LC          1
Name: SourceName, Length: 456, dtype: int64


In [191]:
def filterEdges(g, edge_drop_factor):
    # g = nx.DiGraph(g)
    remove_edges = []
    for source, target, weight in g.edges.data("weight"):
        alarm_count = g.nodes[target]["count"]
        action_count = g.nodes[source]["count"]

        alarms2action_ratio = alarm_count/action_count
        weight_alarm_ratio =  weight/alarm_count

        # if edge_drop_factor * weight < g.nodes[target]["count"]:
        if  alarms2action_ratio <  0.20:
            remove_edges.append((source, target))
        elif alarms2action_ratio <0.10 and weight_alarm_ratio < 0.25:
            remove_edges.append((source, target))

    g.remove_edges_from(remove_edges)
    g.remove_nodes_from(list(nx.isolates(g)))
    return g

In [208]:
# initial values of thresholds:
# first elif : action_alarms_ratio: 0.4  , weight_alarm_ratio: 0.25
# second elif : action_alarms_ratio: 0.3  , weight_alarm_ratio: 0.8

#notes
# decreasing the action_alarms_ratio will increase the number of edges and takes too much time to compute

# tryings:
# attempt_0 = 0.4, 0.25, 0.3, 0.8    # initial values
# attempt_1 : 0.35, 0.25, 0.35, 0.8  # done
# attempt_2 : 0.40, 0.25, 0.3, 0.8   # done, 32 relation
# attempt_3 : 0.45, 0.25, 0.3, 0.8   # done, 32 relation, the output is different than the original one
# attempt_4 : 0.55, 0.25, 0.3, 0.8   # done, 28 relation, the output is different
# attempt_5 : 0.65, 0.25, 0.3, 0.8   # done, 27 relation, the output is different than 4th
# attempt_6 : 0.85, 0.25, 0.3, 0.8   # done, 24 relation, the output is different than 5th
# attempt_6_1 : 0.20, 0.25, 0.3, 0.8  # done, 41 relation, the num. of relation increase because the num. of edges will increase as you decrease the actions_alarm_ratio

# attempt_7 : 0.4, 0.35, 0.3, 0.8    # done, 27 relation, the output is different than 6th
# attempt_8 : 0.4, 0.50, 0.3, 0.8    # done, 20 relation, the output is different than 7th
# attempt_9 : 0.4, 0.70, 0.3, 0.8    # done, 16 relation, the output is different than 8th

# attempt_10 : 0.4, 0.25, 0.5, 0.8   # done, 32 relation, the output is different than 9th and original file
# attempt_11 : 0.4, 0.25, 0.7, 0.8   # done, 32 relation, the output is same with the 10th file because it excessed the threshold.

# attempt_12 : 0.4, 0.25, 0.3, 0.95  # done, 32 relation, the output is same with the 10th and 11th file
# attempt_13 : 0.4, 0.25, 0.3, 0.65  # done, 32 relation, the output is same with the 10th, 11th and 12th file


In [209]:
def foo1(args):

        df_alarms = args[0]
        df_actions =args[1]
        min_intersect = args[2]
        time_to_react = args[3]
        edge_drop_factor = args[4]
        actionAndAlarms = []

        action2alarm = Action2Alarm()

        # print(f'Months in Alarms = {df_alarms["Year-Month"].unique()} \n Months in Actions = {df_actions["Year-Month"].unique()}')

        main_g = action2alarm.getOperatorAlarmRelationGraph(df_alarms=df_alarms, df_actions=df_actions, min_graphs_intersection_filter=min_intersect, time_to_react=time_to_react,edge_drop_factor=edge_drop_factor)
        
            # %%
        # operator_nodes = [action for action in main_g.nodes if action.find("Operator") != -1]
        alarm_nodes = [alarm for alarm in main_g.nodes if alarm.find("Operator") == -1]

        # print(f">> # of Edges Main Graph {main_g.number_of_edges()}")
        # print(f">> Total number of operator Tags in the graph={len(operator_nodes)}")
        # print(f">> Total number of Alarm Tags in the graph = {len(main_g.nodes)- len(operator_nodes)}")

        dict_to_print = {}
        for node in alarm_nodes:
            alarm_count = main_g.nodes[node]['count']
            num_neg = len(list(main_g.predecessors(node)))
            # print(f"{op} Count:{action_count}|| Correspodning Alarms = {num_neg} ", end = " " )
            s = ""
            for op_node in main_g.predecessors(node):
                # print(op_node)
                act_count = main_g.nodes[op_node]['count']
                edge_weight = main_g.edges[(op_node,node)]['weight']
                action_alarms_ratio = round(alarm_count/act_count,2)
                weight_alarm_ratio =  round(edge_weight/alarm_count,2)

                # s = s+ f" {n}(count:{alarm_count})(weight:{edge_weight}), (alarm/action: {action_alarms_ratio}), (alarm/weight: {action_weight_ratio}) "
                # dict_to_print[op] = s 
                op_node = op_node.replace("Operator->","")
                if op_node == node:
                    s = s+ f"(**{op_node}, count:{act_count},weight:{edge_weight}, alarm/action: {action_alarms_ratio}, weight/alarm: {weight_alarm_ratio}) "
                    dict_to_print[node] = s 
                elif action_alarms_ratio > 0.2 and weight_alarm_ratio>0.25:
                    s = s+ f"(2-{op_node}, count:{act_count},weight:{edge_weight}, alarm/action: {action_alarms_ratio}, weight/alarm: {weight_alarm_ratio}) "
                    dict_to_print[node] = s
                elif action_alarms_ratio>0.3 and weight_alarm_ratio > 0.8: # these ratios are not finalized and final ratios will depend on the Ahmet Can feedback
                    s = s+ f"(3-{op_node}, count:{act_count},weight:{edge_weight}, alarm/action: {action_alarms_ratio}, weight/alarm: {weight_alarm_ratio}) "
                    dict_to_print[node] = s  
                # elif action_alarms_ratio >0.5 and action_alarms_ratio <=1:
                #     print(f"{n}(count:{alarm_count})(weight:{edge_weight}), (ratio: {action_alarms_ratio})", end = ", ")


        for alarm_node in dict_to_print.keys():
            alarm_count = main_g.nodes[alarm_node]['count']
            num_neg = len(list(main_g.predecessors(alarm_node)))

            s = f"{alarm_node} Count:{alarm_count}|| Corresponding Actions = {num_neg}, => {dict_to_print[alarm_node]} " 
            print(s)
            actionAndAlarms.append(s)
        
        return actionAndAlarms

In [210]:
attempt_txt = "output_attempt6_1.txt"
attempt_csv = "output_attempt6_1.xlsx"
dir_ = "outputs/"
txt= "txt/"
xlsx = "xlsx/"

In [211]:
#print(f"Total Alarms: {df_temp_alarms.shape}, Total Operator Action: {df_temp_actions.shape}, Total Sources: {len(df_temp_alarms['SourceName'].unique())} ")

# %%
start = time.time()
""" 
    Filter the Alarm Data
    1. Ignore the communication Alarms
    2. Ignore the momentary alarms => 20 seconds
    3. Remove Staling Alarms => 12 hours    
    4. Remove sources which are triggered less 20 in whole dataset
    5. Include all the months
    6. DO SKIP ANY SOURCENAME IF IGNORING COMMUNICATION ALARMS
"""
ignore_comm_alarms :Final = True
momentary_alarms_f:Final = 2*60  # seconds
staling_alarm_f:Final = (60*60) * 4 # hours
min_alarms_per_source_f:Final = 40 # any source which is not triggered atleast 20 times in whole dataset will be removed
months_f:Final = df_temp_alarms["Year-Month"].unique()
print(f">> Unique Year-Month {months_f}")
snames_f:Final = [] # ONLY USE IF NOT IGNORING COMM ALRMS


df_alarms_new = filterAlarmData(df_temp_alarms, months=months_f, sources_filter=snames_f,
                                     monmentarly_filter=momentary_alarms_f, staling_filter=staling_alarm_f, ingore_communication_alarms=ignore_comm_alarms, min_alarms_per_source=min_alarms_per_source_f)

df_actions_new = df_temp_actions[df_temp_actions["Year-Month"].isin(months_f)]

sname2months = {}
for sname in df_alarms_new["SourceName"].unique():
    sname2months[sname] = df_alarms_new[df_alarms_new["SourceName"].isin([sname])]["Year-Month"].unique()



monthCount2Snames = {} 

for key in range(len(list(df_alarms_new["Year-Month"].unique()))):
    monthCount2Snames[key+1] = []

for sname in sname2months.keys():
    monthCount2Snames[len(sname2months[sname])].append(sname)

print(f">> Month Count 2 alarms: {monthCount2Snames}" )

""" 
    For Opertartor Graphs
    1. Construct sub-graphs equal to number of months
    2. Min intersections is sub-graphs/2 + 1
    3. Remove edges which are not significantaly contributing to nodes count
        It will vary in op actions and in grouping. In op actions
        it will be higher but in groupign it will be lower.
"""
# num_sub_graphs:Final = len(months_f)
# min_intersection_f:Final = 2
# edge_filter:Final = 1 # BE CAREFUL OVER HERE
print("Total Time spent ", time.time()-start)


>> Unique Year-Month ['(2020, 3)' '(2019, 7)' '(2019, 8)' '(2019, 3)' '(2019, 10)' '(2019, 11)'
 '(2019, 9)' '(2020, 2)' '(2019, 5)' '(2019, 6)' '(2019, 12)' '(2019, 4)'
 '(2020, 1)']
>>Preprocessing... 
   Months to include=['(2020, 3)' '(2019, 7)' '(2019, 8)' '(2019, 3)' '(2019, 10)' '(2019, 11)'
 '(2019, 9)' '(2020, 2)' '(2019, 5)' '(2019, 6)' '(2019, 12)' '(2019, 4)'
 '(2020, 1)']
  Ignore Sources=[]
  Ingnore Momentarlily Alarms Filter=120seconds 
   Ignoreing Staling Alarms Filter=4.0 hours, 
 Ignore Communication Alarms = True 
 Remove sources whose count is less than 40
>> Month Count 2 alarms: {1: ['47LI900B', '47LI900ABC'], 2: ['47FIC1543', '47LI850C'], 3: ['47TAH808-ANN', '47PI010', '47FIC011', '47LI900C', '47FIC017'], 4: ['47FLI-032', '47FSL014-ANN', '47FLI-041', '47FI1505D', '48XL011-ANN', '47PIC1712', '47PI1734', '47LI606A', '47PI1519', '47FI1567'], 5: ['47USI802', '48UA014-ANN', '47FI1502B', '47FI1502D', '47FI1505B', '47TXI1592', '47FIC1563', '47TI1723', '47LIC1551', '48

In [212]:
start = time.time()
actionAndAlarms =[]
# action2alarm = Action2Alarm()
for sname in  df_alarms_new['SourceName'].unique():
    mc = len(sname2months[sname])
    intersect_threshold = mc-1 # gamma
    if  mc <=5:
        continue
    # print(f">> Source Name {sname}, Alarm Appear in {mc} months, Intersection Threshold ={intersect_threshold}" )
    df1 = df_alarms_new[df_alarms_new['SourceName'].isin([sname])]
    df2 = df_actions_new[df_actions_new['Year-Month'].isin(list(df1['Year-Month'].unique()))] 

    actionAndAlarms += foo1((df1,df2,intersect_threshold,30*60*60, -1))
    # endAlarmTime should be around 1 hour

print("Total Time spent ", time.time()-start)
# for aa in actionAndAlarms:
#     print(f"++{aa}")

# foo1((df_temp_alarms,df_temp_actions,6,4*60*60))

47TI003A Count:2015|| Corresponding Actions = 24, => (2-47TIC274, count:5269,weight:597, alarm/action: 0.38, weight/alarm: 0.3) (2-47FIC1563, count:3903,weight:555, alarm/action: 0.52, weight/alarm: 0.28)  
47FLI-039 Count:4219|| Corresponding Actions = 56, => (2-47FIC013, count:4069,weight:1942, alarm/action: 1.04, weight/alarm: 0.46) (2-47TIC937E, count:3867,weight:1831, alarm/action: 1.09, weight/alarm: 0.43) (2-47TIC274, count:5269,weight:2452, alarm/action: 0.8, weight/alarm: 0.58) (2-47TIC927E, count:2940,weight:1415, alarm/action: 1.44, weight/alarm: 0.34) (2-47TIC007, count:3026,weight:1449, alarm/action: 1.39, weight/alarm: 0.34) (2-47TIC259, count:2908,weight:1357, alarm/action: 1.45, weight/alarm: 0.32) (2-47FIC1563, count:3903,weight:1829, alarm/action: 1.08, weight/alarm: 0.43) (2-47PIC027, count:4674,weight:2126, alarm/action: 0.9, weight/alarm: 0.5)  
47TI003B Count:2733|| Corresponding Actions = 27, => (2-47TIC274, count:5269,weight:801, alarm/action: 0.52, weight/alarm

47LIC1501 Count:631|| Corresponding Actions = 17, => (2-47FIC017, count:2156,weight:330, alarm/action: 0.29, weight/alarm: 0.52) (2-47TIC927E, count:2940,weight:323, alarm/action: 0.21, weight/alarm: 0.51) (2-47TIC007, count:3026,weight:370, alarm/action: 0.21, weight/alarm: 0.59) (2-47TIC259, count:2908,weight:305, alarm/action: 0.22, weight/alarm: 0.48) (2-47PIC017, count:1788,weight:178, alarm/action: 0.35, weight/alarm: 0.28)  
47LIC1518 Count:508|| Corresponding Actions = 3, => (2-47FIC1534, count:1854,weight:851, alarm/action: 0.27, weight/alarm: 1.68) (2-47FIC1542, count:2257,weight:956, alarm/action: 0.23, weight/alarm: 1.88)  
47FI043 Count:1873|| Corresponding Actions = 30, => (2-47FIC013, count:3371,weight:612, alarm/action: 0.56, weight/alarm: 0.33) (2-47TIC937E, count:3114,weight:581, alarm/action: 0.6, weight/alarm: 0.31) (2-47TIC274, count:4400,weight:820, alarm/action: 0.43, weight/alarm: 0.44) (2-47FIC1563, count:3638,weight:659, alarm/action: 0.51, weight/alarm: 0.35)

47LIC1511 Count:111|| Corresponding Actions = 1, => (2-47PIC1504, count:482,weight:94, alarm/action: 0.23, weight/alarm: 0.85)  
47PIC1504 Count:59|| Corresponding Actions = 1, => (**47PIC1504, count:476,weight:180, alarm/action: 0.12, weight/alarm: 3.05)  
47LIC1503 Count:298|| Corresponding Actions = 1, => (2-47FIC1507, count:453,weight:172, alarm/action: 0.66, weight/alarm: 0.58)  
48FFIC019 Count:85|| Corresponding Actions = 1, => (**48FFIC019, count:591,weight:51, alarm/action: 0.14, weight/alarm: 0.6)  
48FFX019 Count:76|| Corresponding Actions = 3, => (2-48BSL006-BYP, count:49,weight:42, alarm/action: 1.55, weight/alarm: 0.55)  
47FSL019-ANN Count:173|| Corresponding Actions = 23, => (2-47PIC028, count:715,weight:72, alarm/action: 0.24, weight/alarm: 0.42)  
Total Time spent  207.8054232597351


In [213]:
print(f"The number of alarm-operator relations for this particular example: {len(actionAndAlarms)}")

The number of alarm-operator relations for this particular example: 41


In [214]:
# for aa in actionAndAlarms:
#     print(aa)
with open(dir_ + txt + attempt_txt,'w') as f:
    for aa in actionAndAlarms:
        f.write(aa)
        f.write("\n")
    # f.writelines(actionAndAlarms)

In [215]:
from sys import prefix


#actionAndAlarms[0].split(" ")[0]  # alarm
#actionAndAlarms[20].split(" ")[7].replace("(","").replace(",","") 
prefixes = ["(**", "(2-", "(3-"]

alarm_action_dict = {}
for i in actionAndAlarms:
    i = i.split(" ")
    alarm_action_dict[i[0]] = [x.replace("(","").replace(",","") for x in i if x.startswith(tuple(prefixes))]
print(alarm_action_dict)

{'47TI003A': ['2-47TIC274', '2-47FIC1563'], '47FLI-039': ['2-47FIC013', '2-47TIC937E', '2-47TIC274', '2-47TIC927E', '2-47TIC007', '2-47TIC259', '2-47FIC1563', '2-47PIC027'], '47TI003B': ['2-47TIC274', '2-47FIC1563'], '47FIC1535': ['**47FIC1535', '2-47FIC1534', '2-47FIC013', '2-47FIC1542', '2-47TIC937E', '2-47TIC274', '2-47TIC927E', '2-47TIC007', '2-47TIC259', '2-47FIC1563', '2-47PIC1507'], '47LIC1555': ['2-47TIC274'], '48XL001-ANN': ['2-47TIC274'], '47PIC1507': ['2-47FIC1563'], '47FLI-042': ['2-47FIC1520', '2-47FIC013', '2-47TIC937E', '2-47TIC274', '2-47TIC927E', '2-47TIC007', '2-47TIC415', '2-47TIC259', '2-47HIC1502', '2-47FIC1563', '2-47HIC1501', '2-47PIC027'], '47FLI-037': ['2-47FIC013', '2-47TIC927E', '2-47HIC1502', '2-47FIC1563'], '48PAL013-ANN': ['2-48TIC034'], '47LIC1520': ['2-47FIC1534'], '47LIC1506': ['2-47FIC1522A'], '47XL016-ANN': ['2-47FIC1534', '2-47FIC017', '2-47FIC1520', '2-47FIC013', '2-47FIC035', '2-47FIC036', '2-47FIC1542', '2-47TIC937E', '2-47TIC274', '2-47TIC927E', 

In [216]:
# import dict to excel
import pandas as pd
df = pd.DataFrame.from_dict(alarm_action_dict, orient='index')
df.to_excel(dir_ + xlsx + attempt_csv)

In [218]:
# is it same with the original function?
file1 = []
file2 = []
with open(dir_ + txt + "output_attempt6.txt") as f:
    for line in f:
        file1.append(line) 
        
with open(dir_ + txt + attempt_txt) as f2:
    for line in f2:
        file2.append(line)
        
if file1==file2:
    print("The file is the same with the original file, nothing changed.")
else:
    print("The file is different from the original file, something has changed.")
    print(len(file2))

The file is different from the original file, something has changed.
41
